## Topic-reduction

The model initially found 229 topics which is too much to interpret - thus, we decided to reduce the number of topics using top2vec's hierarchical topic reduction method.

This notebook contains data-preparation part of topic reduction -- for vizzes please go visit this [repository](https://github.com/hcss-utils/streamlit-topic-reduction)

In [1]:
import umap
import pandas as pd
from top2vec import Top2Vec

In [2]:
def reduce_topics(model, min_topics, max_topics, n_words):
    for num in range(min_topics, max_topics+1):
        model.hierarchical_topic_reduction(num)
        topic_words, word_scores, topic_nums = model.get_topics(reduced=True)
        words = [", ".join(topic_words[topic][:n_words]) for topic in topic_nums]
        mapped = [words[num_label] for num_label in model.doc_top_reduced]
        yield {
            num: {
                "unique_words": words,
                "classification_num": model.doc_top_reduced.tolist(),
                "classification_label": mapped,
            }
        }

In [3]:
model = Top2Vec.load("../models/pq-model")

In [4]:
classifications = reduce_topics(model, min_topics=2, max_topics=20, n_words=5)
mappings = {k:v for d in classifications for k, v in d.items()}

In [5]:
umap_model = umap.UMAP(
    n_neighbors=15,
    n_components=2,
    metric="cosine",
    random_state=42
).fit(model._get_document_vectors(norm=False))

In [6]:
embeddings = pd.DataFrame(umap_model.embedding_, columns=("x", "y"))

In [7]:
for num, values in mappings.items():
    embeddings[f"num_label_{num}"] = mappings.get(num)["classification_num"]
    embeddings[f"word_label_{num}"] = mappings.get(num)["classification_label"]

In [8]:
embeddings

,x,y,num_label_2,word_label_2,num_label_3,word_label_3,num_label_4,word_label_4,num_label_5,word_label_5,...,num_label_16,word_label_16,num_label_17,word_label_17,num_label_18,word_label_18,num_label_19,word_label_19,num_label_20,word_label_20
0,8.063369,-0.291642,0,"pyongyang, hardly, surely, scarcely, inevitably",0,"pyongyang, hardly, sophism, scarcely, surely",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",7,"republicans, republican, gop, democrat, democrats"
1,8.119796,-0.541421,0,"pyongyang, hardly, surely, scarcely, inevitably",0,"pyongyang, hardly, sophism, scarcely, surely",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",6,"republicans, republican, gop, democrat, democrats",7,"republicans, republican, gop, democrat, democrats"
2,10.569654,5.105814,0,"pyongyang, hardly, surely, scarcely, inevitably",0,"pyongyang, hardly, sophism, scarcely, surely",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,7,"nato, russia, enlargement, russian, european",8,"nato, russia, enlargement, russian, moscow",8,"nato, russia, enlargement, russian, moscow",7,"nato, russia, enlargement, russian, moscow",6,"nato, russia, enlargement, russian, moscow"
3,11.484039,5.563009,0,"pyongyang, hardly, surely, scarcely, inevitably",0,"pyongyang, hardly, sophism, scarcely, surely",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,7,"nato, russia, enlargement, russian, european",8,"nato, russia, enlargement, russian, moscow",8,"nato, russia, enlargement, russian, moscow",7,"nato, russia, enlargement, russian, moscow",6,"nato, russia, enlargement, russian, moscow"
4,11.507264,5.589423,1,"airmen, warfighter, dod, cyber, sustainment",1,"airmen, warfighter, dod, cyber, sustainment",1,"airmen, warfighter, dod, cyber, sustainment",1,"airmen, warfighter, dod, cyber, sustainment",...,13,"cyber, malicious, cyberattacks, cybersecurity,...",13,"cyber, malicious, cyberattacks, cybersecurity,...",13,"cyber, malicious, cyberattacks, cybersecurity,...",13,"cyber, malicious, cyberattacks, cybersecurity,...",13,"cyber, malicious, cyberattacks, cybersecurity,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26520,9.129981,4.927581,1,"airmen, warfighter, dod, cyber, sustainment",1,"airmen, warfighter, dod, cyber, sustainment",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,7,"nato, russia, enlargement, russian, european",8,"nato, russia, enlargement, russian, moscow",8,"nato, russia, enlargement, russian, moscow",7,"nato, russia, enlargement, russian, moscow",6,"nato, russia, enlargement, russian, moscow"
26521,8.797188,4.527788,0,"pyongyang, hardly, surely, scarcely, inevitably",0,"pyongyang, hardly, sophism, scarcely, surely",0,"hardly, scarcely, surely, fashionable, inevitably",0,"scarcely, hardly, surely, inevitably, europeans",...,7,"nato, russia, enlargement, russian, european",8,"nato, russia, enlargement, russian, moscow",8,"nato, russia, enlargement, russian, moscow",7,"nato, russia, enlargement, russian, moscow",6,"nato, russia, enlargement, russian, moscow"
26522,9.312355,-0.398733,0,"pyongyang, hardly, surely, scarcely, inevitably",2,"arsenals, arsenal, reductions, weapons, abm",2,"arsenals, reductions, arsenal, abm, weapons",2,"arsenals, reductions, arsenal, abm, weapons",...,7,"nato, russia, enlargement, russian, european",5,"arsenals, arsenal,

In [9]:
embeddings.to_csv("../data/processed/deciding-on-topic-reduction.csv", index=False)